In [1]:
import pandas as pd, numpy as np, json, os

In [188]:
m=json.loads(open('json/member_timelines.json','r').read())

In [330]:
data={'Ország':{},'Bizottság':{},'Párt':{}}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    if i['Típus'] in list(data.keys()):
        if i['Csoportok'] not in data[i['Típus']]:data[i['Típus']][i['Csoportok']]={}
        for t in pd.date_range(i['start'],i['end']):
            st=str(t)[:10]
            if st not in data[i['Típus']][i['Csoportok']]:data[i['Típus']][i['Csoportok']][st]=set()
            data[i['Típus']][i['Csoportok']][st].add(i['Name'])

0.0 %
3.289040915668991 %
6.578081831337982 %
9.867122747006974 %
13.156163662675963 %
16.445204578344956 %
19.734245494013948 %
23.023286409682935 %
26.312327325351927 %
29.60136824102092 %
32.89040915668991 %
36.1794500723589 %
39.468490988027895 %
42.75753190369688 %
46.04657281936587 %
49.33561373503486 %
52.62465465070385 %
55.913695566372844 %
59.20273648204184 %
62.49177739771083 %
65.78081831337983 %
69.06985922904882 %
72.3589001447178 %
75.64794106038678 %
78.93698197605579 %
82.22602289172477 %
85.51506380739376 %
88.80410472306276 %
92.09314563873174 %
95.38218655440073 %
98.67122747006972 %


In [339]:
data2=dict(data)
for tipus in data:
    for csoport in data[tipus]:
        for t in data[tipus][csoport]:
            data2[tipus][csoport][t]=list(data[tipus][csoport][t])

In [340]:
#save
open('E:/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pairs.json','w').write(json.dumps(data2))

536767417

In [4]:
#load
# data=json.loads(open('E:/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pairs.json','r').read())
data=json.loads(open('C:/Users/csala/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pairs.json','r').read())

In [ ]:
network={}
tnetwork={}
weights={'Ország':2,'Bizottság':3,'Párt':1}
for tipus in data:
    for csoport in data[tipus]:
        print(csoport)
        for t in data[tipus][csoport]:
            names=list(data[tipus][csoport][t])
            if t not in tnetwork:tnetwork[t]={}
            #linear time anneal
            anneal=(pd.to_datetime('2019-05-01')-pd.to_datetime(t))/(pd.to_datetime('2019-05-01')-pd.to_datetime('1990-01-01'))
            for r in range(len(names)):
                name=names[r]
                if name not in network: network[name]={}
                if name not in tnetwork[t]: tnetwork[t][name]={}
                for p in range(len(names)):
                    name2=names[p]
                    if name!=name2:
                        if name2 not in network[name]: network[name][name2]=0
                        network[name][name2]+=weights[tipus]*anneal
                        if name2 not in tnetwork[t][name]: tnetwork[t][name][name2]=0
                        tnetwork[t][name][name2]+=weights[tipus]*anneal

🇫🇷 Franciaország
🇬🇷 Görögország
🇵🇭 Fülöp-szigetek
🇦🇺 Ausztrália
🇧🇪 Belgium
🇧🇾 Fehéroroszország
🇨🇴 Kolumbia
🇫🇮 Finnország
🇮🇩 Indonézia
🇰🇷 Dél-Korea
🇲🇽 Mexikó
🇵🇦 Panama
🇸🇰 Szlovákia
🇻🇪 Venezuela
🇨🇱 Chile
🇸🇾 Szíria
🇦🇷 Argentína


In [ ]:
open('json/member_network_raw.json','w').write(json.dumps(network))

Format for D3plus

Normalize to fraction of all committee days spent together

In [ ]:
member_rings_normed=[]
min_ratio=4
names={}
all_names=set()
for iname,name in enumerate(network):
    if iname%100==0: print(iname/len(network)*100,'%')
    for name2 in network[name]:
        if name!=name2:
            #if not already processed
            insert=False
            if name2 not in names:
                insert=True
            elif name not in names[name2]:
                insert=True
            if insert:
                #if partner is important for both
                ratio=network[name][name2]/np.median(list(network[name].values()))
                ratio2=network[name2][name]/np.median(list(network[name2].values()))
                if ((ratio>min_ratio) and (ratio2>min_ratio)):
                    avg_ratio=(ratio+ratio2)/2
                    if name not in names: names[name]=set()
                    names[name].add(name2)
                    all_names.add(name)
                    all_names.add(name2)
                    member_rings_normed.append({'source':name,'target':name2,'ratio':avg_ratio})

In [ ]:
print(len(member_rings_normed))

In [ ]:
#add lone nodes
member_rings_normed2=member_rings_normed+[{'source':i,'target':'Nincsenek erős kapcsolatok '+i,'ratio':1} for i in network if i not in all_names]

Network analysis

In [ ]:
import networkx as nx

In [ ]:
G=nx.Graph(network) 
pr = nx.pagerank(G)
sorted_pr = sorted(pr.items(), key=lambda kv: kv[1], reverse=True)
member_rings_nodes={i[0]:i[1] for i in sorted_pr}
member_rings_nodes={i:[{'id':'','Befolyás':member_rings_nodes[i]/np.median(list(member_rings_nodes.values()))}] for i in member_rings_nodes}

In [ ]:
tmember_rings_nodes=[]
for t in tnetwork:
    G=nx.Graph(tnetwork[t]) 
    pr = nx.pagerank(G)
    sorted_pr = sorted(pr.items(), key=lambda kv: kv[1], reverse=True)
    smember_rings_nodes={i[0]:i[1] for i in sorted_pr}
    tmember_rings_nodes+=[{'id':i,'x':t,'y':smember_rings_nodes[i]/np.median(list(smember_rings_nodes.values()))} for i in smember_rings_nodes]

In [ ]:
tmember_rings_dict={}
for i in tmember_rings_nodes:
    if i not in tmember_rings_dict:tmember_rings_dict[i]=[]
    z=dict(i)
    z['id']=''    
    tmember_rings_dict[i].append(z)

In [ ]:
open('json/member_rings_normed.json','w').write(json.dumps({'nodes_abs':member_rings_nodes,'nodes_t':tmember_rings_dict,'edges':member_rings_normed2}))